<a href="https://colab.research.google.com/github/SNMHZ/Drug_Recommendation/blob/master/model/MODEL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
import pandas as pd
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import  cosine_similarity

df_train = pd.read_csv("https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/dataset/lem_train.csv", parse_dates=["date"], infer_datetime_format=True)
df_test = pd.read_csv("https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/dataset/lem_test.csv", parse_dates=["date"], infer_datetime_format=True)
df_train = pd.concat([df_train,df_test])

print(type(df_train['review']))

print(df_train.isnull().values.any())
print(df_test.isnull().values.any())

print(len(df_train))
print(len(df_test))

df_train = df_train.dropna(how='any')
df_test = df_test.dropna(how='any')

print(df_train.isnull().values.any())
print(df_test.isnull().values.any())

print(len(df_train))
print(len(df_test))
print(df_train['review'][0])

df_train['condition2']=df_train['condition'].apply(lambda x: re.sub('[^a-zA-Z]',' ',x))
df_test['condition2']=df_test['condition'].apply(lambda x: re.sub('[^a-zA-Z]',' ',x))

In [ ]:
concon=pd.read_csv('https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/dataset/spaced_conidtion.csv', index_col=0)
ori_con=concon['original'].unique()
mod_con=concon['modified'].unique()

for i in range(len(ori_con)):
    ori_con[i]=re.sub('[^a-zA-Z]',' ',ori_con[i])

for i in range(len(mod_con)):
    mod_con[i]=re.sub('[^a-zA-Z]',' ',mod_con[i])

con_dict={}
for i in range(len(mod_con)):
    con_dict[ori_con[i]]=mod_con[i]

print(con_dict)

In [ ]:
con_re = re.compile('(%s)' % '|'.join(con_dict.keys()))

def expand_con(s, con_dict=con_dict):
    def replace(match):
        return con_dict[match.group(0)]
    return con_re.sub(replace, s)

df_train['new'] = df_train['review'].str.cat(df_train['condition2'], sep=' ')
df_train['new'] = df_train['new'].str.lower()

df_train['new100'] = df_train['new'].apply(lambda x: expand_con(x))

In [ ]:
tokenize_data2 = [word_tokenize(sentence) for sentence in df_train['new100']]

print(tokenize_data2[:2])

model2 = Word2Vec(sentences= tokenize_data2, size=100, window=5, min_count=5, workers=4, sg=0)

In [ ]:
df_train['condition3'] = df_train['condition2'].str.lower().apply(lambda x: expand_con(x))
df_test['condition3'] = df_test['condition2'].str.lower().apply(lambda x: expand_con(x))
cond_norm=df_train['condition3'].value_counts(True)

cond_norm['birthcontrol']

In [ ]:
def predictCondition(data):
    result = {}
    # print(words)

    for word in data:
        try:
            tmp = model2.wv.most_similar(word, topn=100)
            print(word)
            # print(len(tmp))
            # print(tmp)
            result[word]= tmp
        except:
            print(word + " not in here\n")
    
    return result

def predictCondition2dict(_predictCondition):
  predict_dict = {}
  for key, value in _predictCondition:
    predict_dict[key] = value
  return predict_dict

def predictCondition_concat(_dict_list):
  sum_dict = {}
  for _dict in _dict_list:
    for key in _dict.keys():
      if key in mod_con:
        if key in sum_dict:
          #print(_dict[key])
          sum_dict[key]+=(_dict[key]*cond_norm[key])
        else:
          #print(key, _dict[key])
          sum_dict[key]=(_dict[key]*cond_norm[key])
          #print(sum_dict[key])
  return sum_dict

def predictConditionSum(data):
  predict_res = predictCondition(data)
  m_list = []
  for key in predict_res:
    m_list.append(predictCondition2dict(predict_res[key]))
  return predictCondition_concat(m_list)

from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

n=WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def lemlem(msg):
  msg=msg.replace("&#039;", "")
  msg=msg.replace(r'[^\w\d\s]',' ')
  msg=re.sub('[^a-zA-Z]',' ',msg)
  low_msg = msg.lower().split()
  stop_msg=[]
  for w in low_msg: 
    if w not in stop_words: 
        stop_msg.append(w) 
  result=[n.lemmatize(w) for w in stop_msg]
  return result

def isincheck(_str):
  if len(set(only_condition.isin([_str.lower()])))==2:
    return True
  return False

def findDrugName(condition):

    tempIndex = df_train['condition3'] == condition.lower()
    # 해당 컨디션 단어와 완전 일치하는 경우 약이 매칭 되도록 함
    #tempIndex = df_all['condition_lower'].str.contains(condition.lower())
    # 해당 컨디션 단어가 들어만 가도 약이 매칭 되도록 함
    #print("data type is = ", type(df_all))
    
    tempSet = df_train[['condition', 'drugName', 'rating']][tempIndex]

    result = tempSet.sort_values(by=['rating'], axis=0, ascending=False)
    result=result.groupby(['drugName'], as_index=False).mean()
    result=result.sort_values(by=['rating'], axis=0, ascending=False)

    print("condition is ", condition, '\n',result['drugName'][:5],"\n=====================")

In [ ]:
while True:
    msg = input("data : ")

    tmp = predictConditionSum(lemlem(msg))

    m_sorted_res = sorted(list(tmp.items()), key=lambda x:x[1], reverse=True)[:1000]

    for j in m_sorted_res[:10]:
      findDrugName(j[0])

    #only_condition = pd.concat([df_train['condition3'], df_test['condition3']], axis=0)
    #only_condition = only_condition.drop_duplicates()
    #for i in only_condition.index:
    #    only_condition[i] = only_condition[i].lower()
    #for i, res in enumerate(m_sorted_res):
    #    if isincheck(res[0]):
    #        print(i+1, res)